# Praktikum Informationstechnik im Maschinenbau II
## P06 - Berechnung von Stabkräften in einem Fachwerk mit numpy

**Diese Aufgabe erfordert umfassende Kenntnis der Übung 01 (ausführlich kommentierte Musterlösung)**

Im Bild ist ein dreidimensionales Stabwerk bestehend aus den Knoten (I bis V) und Stäben (1 - 9) jeweils zwischen zwei Knoten dargestellt. Über sechs weitere Stäbe (A - F) ist die Struktur mit der ruhenden Umgebung verbunden (Auflagerkräfte). An der Spitze der Struktur wirkt eine beliebig orientierte äußere Kraft $\vec{F}$, die durch ihre Komponenten in x, -y und -z-Richtung $F_{1}$, $F_{2}$ und $F_{3}$ gegeben ist.



![](bilder/3d_trusses_task.gif)   ![](bilder/3d_trusses_geometry.gif)

Das rechte Bild definiert das Koordinatensystem und die Positionen der Knoten I bis V. 

>Bitte beachten. Die z-Koordinate des Knotens II über der Grundfläche beträgt $3a$ und ist im linken Bild etwas versteckt

In diesem Praktikum sollen Sie mit Hilfe von numpy die Beträge der Kräfte in den Stäben $F_{S1}$ bis $F_{S9}$ und die Auflagerkräfte $F_A$ bis $F_F$ für einen beliebigen Lastfall berechnen. 

## Lösungsweg

Sie müssen, so wie es in der Übung gezeigt wurde, für jeden Knoten das vektorielle Kräftegleichgewicht formulieren. Siehe Bild

![](bilder/3d_trusses_equilibrium.gif) 

Die Gleichungen für alle 5 Knoten kombinieren Sie zu einem Gleichungssystem aus 15 Gleichungen für 15 Unbekannte in der Form, die zur Lösung des Gleichungssystems mit numpy notwendig ist
$$
\mathbf{A} \vec{x} = \vec{y}
$$
Darin ist $\mathbf{A}$ eine Koeffizientenmatrix, $\vec{x}$ der Vektor aller Stab- und Auflagerkräfte und $\vec{y}$ die rechte Gleichungsseite, die aus den äußeren Kräften gebildet wird.

Wir vereinbaren, dass eine Zugkraft entsprechend dem Bild positiv gezählt wird. 

## Vorgehensweise

Gehen Sie in den folgenden Schritten vor:

- a) Lesen Sie aus der Geometrie des Stabwerkes die Ortsvektoren der Knoten $\vec{P}_1$ (Knoten I) bis $\vec{P}_5$ (Knoten V) ab
- b) Berechnen Sie alle Richtungsvektoren zwischen jeweils zwei Knoten
- c) Formulieren Sie das Gleichungssystem für Knoten I in der Form $\mathbf{A}_1 \cdot \vec{x} = \vec{y_1}$
- d) Formulieren Sie die Gleichungssysteme für Knoten II bis V in analoger Weise (für Lastfall $\vec{F}=\begin{bmatrix}0 & 0 & -1\end{bmatrix}^T$, Druckkraft von oben)
- e) Kombinieren Sie die Teilsysteme zum Gesamtsystem $\mathbf{A} \cdot \vec{x} = \vec{y}$ (15 Gleichungen für 15 Unbekannte)
- f) Lösen Sie das Systeme mit Hilfe von `numpy.linalg.solve()` 
- g) Prüfen und interpretieren Sie die Lösung
- h) Untersuchen Sie weitere Lastfälle

>Anmerkung: als `A.npy` ist eine Numpy-Datei beigefügt, die die $\mathbf{A}$-Matrix als Ergebnis aus Aufgabenteil e) beinhaltet
    

## a) Ortsvektoren der Knoten $\vec{P}_1$ (Knoten I) bis $\vec{P}_5$ (Knoten V)

In [1]:
import numpy as np
A=np.load('A.npy')

In [2]:
# Alle 5 Ortsvektoren
a=1  
P1 = np.array([[0, 0, 0]]).T
P2 = np.array([[a, 1.5*a, 3*a]]).T
P3 = np.array([[0, 3*a, 0]]).T
P4 = np.array([[2*a, 0, 0]]).T
P5 = np.array([[2*a, 3*a, 0]]).T




## b) Richtungsvektoren $r_{ij}$ von $P_i$ $P_j$

In [3]:
# Definition einer Funktion zur Berechnung eines normierten Richtungsvektors zwischen zwei Knoten
def rtgvec(Pi, Pj):
    r=Pj - Pi
    return r / np.linalg.norm(r)

In [4]:
# Test mit dem Richtungsvektor zwischen Knoten I und Knoten V
r15 = rtgvec(P1, P5)
r15

array([[0.5547002 ],
       [0.83205029],
       [0.        ]])

In [5]:
# Anwenden der Function auf alle Knotenverbindungen

# von Knoten I ausgehend
r12 = rtgvec(P1, P2)
r13 = rtgvec(P1, P3)
r14 = rtgvec(P1, P4)
# Gegenrichtung
r21 = - r12
r31 = - r13
r41 = - r14
r51 = - r15

# von Knoten II ausgehend
r23 = rtgvec(P2, P3)
r24 = rtgvec(P2, P4)
r25 = rtgvec(P2, P5)

r32 = - r23
r42 = - r24
r52 = - r25

# von Knoten III ausgehend
r34 = rtgvec(P3, P4)
r35 = rtgvec(P3, P5)

r43 = -r34
r53 = -r35

# von Knoten IV ausgehend
r45 = rtgvec(P4, P5)
r54 = -r45


## c) Gleichungssystem für Knoten I in der Form $\mathbf{A}_1 \cdot \vec{x} = \vec{y_1}$

### Lösungshinweise

Die Kräftebilanz an Knoten I lautet

$$
\vec{F_{S1}} + \vec{F_{S6}} + \vec{F_{S8}} + \vec{F_{S9}} + \vec{F_A} \stackrel{!}{=} \vec{0}
$$
Jede Stab- und Lagerkraft muss in der Form "Größe mal Richtung" (z.B.: $\vec{F_{S6}} = F_{S6}\cdot \vec{r}_{15}$) eingesetzt werden. Für die Stäbe nutzt man die berechneten Richtungsvektoren $\vec{r}_{ij}$. Die Beträge der Lagerkräfte sind mit den jeweiligen Einheitsvektoren $\vec{e}_x$, $\vec{e}_y$, $\vec{e}_z$ zu multiplizieren. Man erhält
$$
\vec{r}_{12}\cdot F_{S1} + \vec{r}_{15}\cdot F_{S6} + \vec{r}_{14}\cdot F_{S8} + \vec{r}_{13}\cdot F_{S9} + (-\vec{e_z})\cdot F_A \stackrel{!}{=} \vec{0}
$$
Um die Teilsysteme zum Gesamtsystem kombinieren zu können, muss der Vektor $\vec{x}$ die Beträge aller Stab- Lagerkräfte beinhalten. Er muss also lauten
$$
\vec{x} = \big[F_{S1}, F_{S2}, \dots F_{S9}, F_A, F_B \dots F_F\big]^T 
$$
Deshalb muss für alle nicht in der Kräftebilanz an Knoten I auftauchenden Kräfte jeweils ein Produkt der Form $\vec{0}\cdot F_{Sx} = \vec{0}$ eingesetzt werden. So vervollständigt lautet die Kräftebilanz:
$$
\vec{r}_{12}\cdot F_{S1} + \vec{0}\cdot F_{S2} + \vec{0}\cdot F_{S3} + \vec{0}\cdot F_{S4} + 
\vec{0}\cdot F_{S5} +  \vec{r}_{15}\cdot F_{S6} + \vec{0}\cdot F_{S7} + \vec{r}_{14}\cdot F_{S8} + \vec{r}_{13}\cdot F_{S9} + 
(-\vec{e_z})\cdot F_A + \vec{0}\cdot F_B + \vec{0}\cdot F_C + \vec{0}\cdot F_E + \vec{0}\cdot F_F\stackrel{!}{=} \vec{0}
$$
Diese wird zur Matrixgleichung
$$
\underbrace{
\begin{bmatrix}\vec{r}_{12} & \vec{0} & \vec{0} & \vec{0} & \vec{0} & \vec{r}_{15} & \vec{0} & \vec{r}_{14} & \vec{r}_{13} & -\vec{e}_z & \vec{0} & \vec{0} & \vec{0} & \vec{0} & \vec{0} \end{bmatrix}
}_{\mathbf{A}_1}
\underbrace{
\cdot \begin{bmatrix} F_{S1} \\ F_{S2} \\ \dots \\ F_{S9} \\ F_A \\ F_B \\ \dots \\F_F\end{bmatrix} 
}_\vec{x} = \underbrace{\begin{bmatrix}0 \\ 0 \\ 0\end{bmatrix}}_{\vec{y}_1}
$$
>Anmerkung: Es handelt sich um ein System aus 3 Gleichungen für 15 Unbekannte. $(3 \times 15) \cdot (15 \times 1) = (3 \times 1)$. $\vec{y}_1$ ist der Vektor aus den Richtungskomponenten der an Knoten I angreifenden äußeren Kraft, hier also $\vec{0}$

In [6]:
# Nullvektor
nv = np.array([[0, 0, 0]]).T

# Einheitsvektoren
ex = np.array([[1, 0, 0]]).T
ey = np.array([[0, 1, 0]]).T
ez = np.array([[0, 0, 1]]).T


In [7]:
# Matrix A1 aus Spaltenvektoren zusammenbauen

# kleine "Montagehilfe" (damit kein Element vergessen wird. Kopieren Sie sich diese Zeilen auch in Aufgabenteil d)
#                 F1   F2   F3   F4   F5   F6   F7   F8   F9      FA   FB   FC   FD   FE   FF
#Ax = np.hstack([ nv,  nv,  nv,  nv,  nv,  nv,  nv,  nv,  nv,     nv,  nv,  nv,  nv,  nv,  nv])
A1 = np.hstack([r12, nv, nv, nv, nv, r15, nv, r14, r13, ez, nv, nv, nv, nv, nv])

In [8]:
# Vektor der rechten Seite
y1 = nv


## d) Gleichungssysteme für Knoten II bis V (Lastfall $\vec{F}=\begin{bmatrix}0 & 0 & -1\end{bmatrix}^T$)

![](bilder/3d_trusses_equilibrium.gif) 

Aufstellen der Gleichungssysteme für die Knoten II bis V

In [9]:
#               F1   F2   F3   F4   F5   F6  F7 F8  F9  FA  FB  FC  FD  FE  FF
A2 = np.hstack([r12, r24, r25, r23, nv, nv, nv, nv, nv, nv, nv, nv, nv, nv, nv])

F=np.array([[0, 0, -1]]).T
y2 = -F

In [10]:
#                 F1   F2   F3   F4    F5   F6   F7   F8   F9    FA   FB   FC   FD   FE   FF
A3 = np.hstack([ nv,  nv,  nv,  r32,  r35,  nv,  nv,  nv,  r31, nv,  nv,  nv,  nv,  ey,  -ex])

y3 = nv

In [11]:
#                 F1   F2   F3   F4    F5   F6   F7   F8   F9    FA   FB   FC   FD   FE   FF
A4 = np.hstack([ nv,  r42,  nv,  nv,  nv,  nv,  r45,  r41,  nv, nv,  -ez,  nv,  nv,  nv,  nv])

y4 = nv

In [12]:
#                 F1   F2   F3   F4    F5   F6   F7   F8   F9    FA   FB   FC   FD   FE   FF
A5 = np.hstack([ nv,  nv,  r52,  nv,  r53,  r51,  r54,  nv,  nv, nv,  nv,  -ez,  ey,  nv,  nv])

y5 = nv

## e) Kombinieren zum Gesamtsystem $\mathbf{A} \cdot \vec{x} = \vec{y}$

Keines der Teilsysteme ist für sich lösbar. Fügt man diese jedoch in der folgenden Weise zusammen
$$
\underbrace{\begin{bmatrix} \mathbf{A}_1 \\ \mathbf{A}_2 \\ \mathbf{A}_3 \\ \mathbf{A}_4 \\ \mathbf{A}_5\end{bmatrix}}_{\mathbf{A}}
\cdot \underbrace{
\begin{bmatrix} F_{S1} \\ F_{S2} \\ \dots \\ F_{S9} \\ F_A \\ F_B \\ \dots \\F_F\end{bmatrix} 
}_\vec{x} = \underbrace{\begin{bmatrix} \vec{y}_1 \\ \vec{y}_2 \\ \vec{y}_3 \\ \vec{y}_4 \\ \vec{y}_5\end{bmatrix}}_{\vec{y}}
$$
so entsteht ein (lösbares) System aus 15 Gleichungen für 15 Unbekannte.

In [14]:
# Bauen der A-Matrix des Gesamtsystems
A = np.vstack([A1, A2, A3, A4, A5])

In [15]:
# Vergleich mit Muster-A-Matrix
Aref = np.load('A.npy')
# Ausdruck, der True ergibt, wenn alle Element stimmen
(A == Aref).all()
y1, y2, y3, y4, y5

(array([[0],
        [0],
        [0]]),
 array([[0],
        [0],
        [1]]),
 array([[0],
        [0],
        [0]]),
 array([[0],
        [0],
        [0]]),
 array([[0],
        [0],
        [0]]))

In [16]:
# Bauen des y-Vektors des Gesamtsystems
y = np.vstack([y1, y2, y3, y4, y5])

## f) Lösen des Gleichungssystems 

In [19]:
# Lösen
x = np.linalg.solve(Aref, y)
x

array([[-5.83333333e-01],
       [ 4.62592927e-18],
       [-5.83333333e-01],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 3.00462606e-01],
       [ 0.00000000e+00],
       [-0.00000000e+00],
       [ 4.16333634e-17],
       [-5.00000000e-01],
       [ 0.00000000e+00],
       [-5.00000000e-01],
       [ 4.16333634e-17],
       [ 4.16333634e-17],
       [-0.00000000e+00]])

In [20]:
# Übersicht schaffen: Nahe-Null-Werte durch 0 ersetzen
x[np.isclose(x,0)]=0
x

array([[-0.58333333],
       [ 0.        ],
       [-0.58333333],
       [ 0.        ],
       [ 0.        ],
       [ 0.30046261],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.5       ],
       [ 0.        ],
       [-0.5       ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

## g) Prüfung der Lösung

In [21]:
# Rechnerische Prüfung
np.isclose(A @ x - y,0).all()

<function ndarray.all>

In [ ]:
# Exemplarische Mechanische Prüfung: Kräftebilanz an Knoten III


## h) Lösen für den Lastfall $\vec{F}=\frac{1}{\sqrt{3}}\begin{bmatrix}-1 & 1 & 1\end{bmatrix}^T$ (Zugkraft schräg nach oben)

In [ ]:
F = 
# Rechte Seite
y = 

In [ ]:
x =


In [ ]:
# Rechnerische Prüfung


In [ ]:
# Exemplarische Mechanische Prüfung: Kräftebilanz an Knoten III
